In [ ]:
elastic_rods_dir = '../../../../elastic_rods/python/'
weaving_dir = '../../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_max_distance_to_target_surface, contact_optimization, show_selected_joints, highlight_rod_and_joint)

import vis.fields
import matplotlib.cm as cm
import time

In [ ]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [ ]:
scale = 1
rod_length = 0.07244622391567243 * scale
width = rod_length / 20 * 10 
thickness = width / 10 * 0.35 
print(width, thickness)
io = InputOrganizer('kleinbottle_projected_{}'.format(scale), thickness, width, weaving_dir)

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 1
sw = 10
drw = 0.1
dsw = 0.1

In [ ]:
# vf = show_selected_joints(curved_linkage, [30, 205, 91, 303, 525, 462, 496, 152, 425, 254, 391, 348, 151, 255, 255, 349, 349, 31, 92, 461, 92, 461, 151])
# curved_linkage_view.update(vectorField = vf)

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, interleaving_type=InterleavingType.weaving, use_constant_width = False, width_scale = [0.5, 1.5])
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewerWithSurface(curved_linkage, io.SURFACE_PATH)
set_surface_view_options(curved_linkage_view)
curved_linkage_view.show()

In [ ]:
precomputed_design_parameter = np.load('klein_bottle_optimized_design_parameter_23062020.npy')
precomputed_design_parameter[curved_linkage.numRestKappaVars():] = scale * precomputed_design_parameter[curved_linkage.numRestKappaVars():]

In [ ]:
# rod_view = highlight_rod_and_joint(curved_linkage, 0, 20, io.RIBBON_CS)
# rod_view.show()

In [ ]:
curved_linkage.setExtendedDoFsPSRL(np.load('klein_bottle_optimized_dof_23062020.npy'))

In [ ]:
# curved_linkage.setExtendedDoFsPSRL(np.load('klein_bottle_straight_ribbon_dof_020720.npy'))

In [ ]:
# curved_linkage.setExtendedDoFsPSRL(np.load('/Users/yren/Develop/EPFL_LGG/weaving/weaving/weaving_editor/examples/kleinbottle_nature_figure3/version_2_kleinbottle_nature_figure3_plot_data/stage_1/763_dof.npy'))

In [ ]:
# curved_linkage.setDesignParameters(precomputed_design_parameter)

In [ ]:
get_max_distance_to_target_surface(curved_linkage)

In [ ]:
# ribbons = order_segments_by_ribbons(curved_linkage)
# _ = write_linkage_ribbon_output_florin(curved_linkage, ribbons, io.SUBDIVISION_RESOLUTION, 'straight_kleinbottle', False, 1, distance_scale = 0.05878262638219195)

In [ ]:
# distance_to_surface = np.array(curved_linkage.get_squared_distance_to_target_surface((curved_linkage.visualizationGeometry(averagedMaterialFrames = True, averagedCrossSections = True)[0]).flatten()))
# distance_to_surface = np.sqrt(distance_to_surface)
# cmap = plt.cm.viridis
# colors = cmap(np.array(distance_to_surface)/max(distance_to_surface))
# print(colors)

# rgb_scale = 255
# cmyk_scale = 100
# def cmyk_to_rgb(c,m,y,k):
#     """
#     """
#     r = rgb_scale*(1.0-(c+k)/float(cmyk_scale))
#     g = rgb_scale*(1.0-(m+k)/float(cmyk_scale))
#     b = rgb_scale*(1.0-(y+k)/float(cmyk_scale))
#     return r,g,b
# cmyk_colors = distance_to_surface / max(distance_to_surface) * 100
# colors = [list(cmyk_to_rgb(cmyk_colors[i], 33, 0, 0)) + [0] for i in range(len(distance_to_surface))]
# # ribbon_distance_scale = 0.66/np.max(straight_distance_to_surface)
# # print(ribbon_distance_scale)
# # straight_distance_to_surface *= 0.66/np.max(straight_distance_to_surface)  
# export_linkage_geometry_to_obj(curved_linkage, 'kleinbottle_straight_linkage.obj', distance_to_surface, use_color=True, colors = colors)


In [ ]:
curved_linkage.attraction_weight = 1e-5

In [ ]:
# iterateData = stage_1_optimization(curved_linkage, rw, sw, curved_linkage_view)

In [ ]:
def eqm_callback(prob, i):
    curved_linkage_view.update()

In [ ]:
OPTS.niter = 10000

In [ ]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)

In [ ]:
# np.save('klein_bottle_straight_ribbon_design_parameters_020720.npy', curved_linkage.getDesignParameters())

In [ ]:
get_max_distance_to_target_surface(curved_linkage)

In [ ]:
np.load('distance_to_surface_scale.npy')

In [ ]:
# np.save('distance_to_surface_scale.npy', get_max_distance_to_target_surface(curved_linkage))

In [ ]:
deviation, deviation_vector, joint_normals, joint_projection_normals = get_normal_deviation(curved_linkage)

In [ ]:
set_joint_vector_field(curved_linkage, curved_linkage_view, deviation_vector)

In [ ]:
np.mean(deviation)

In [ ]:
curved_linkage.attraction_weight = 1e2

In [ ]:
structure_view = get_structure_analysis_view(curved_linkage)
structure_view.show()

In [ ]:
optimizer = initialize_stage_2_optimizer(curved_linkage, io.SURFACE_PATH, curved_save_tgt_joint_pos, curved_linkage_view)

In [ ]:
optimizer, opt_iterateData = stage_2_optimization(optimizer, curved_linkage, io.SURFACE_PATH, curved_save_tgt_joint_pos, curved_linkage_view, -1, 0, 2)